# ROI USING THE NAIVE TECHNIQUE OF DIFFERENCE BETWEEN SATURATION AND BRIGHTNESS

In [1]:
import os
import numpy as np

def load_images(folder, size):
    images = []
    rois = []

    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        
        if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue

        try:
            img = cv2.imread(file_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

            # Calculating difference between saturation and brightness
            diff = cv2.absdiff(img[:, :, 1], img[:, :, 2])
            thresh = cv2.threshold(diff, 20, 255, cv2.THRESH_BINARY)[1]
            thresh = cv2.medianBlur(thresh, 5)

            # Finding contours
            contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            # Selecting largest contour as potential ROI
            largest_contour = None
            largest_area = 0
            for cnt in contours:
                area = cv2.contourArea(cnt)
                if area > largest_area:
                    largest_contour = cnt
                    largest_area = area

            # Extracting ROI using mask if largest contour found
            if largest_contour is not None:
                mask = np.zeros_like(img[:, :, 0], dtype=np.uint8)
                cv2.drawContours(mask, [largest_contour], -1, 255, -1)
                img = cv2.bitwise_and(img, img, mask=mask)

                # Resize image and ROI to specified size
                img = cv2.cvtColor(img, cv2.COLOR_HSV2BGR)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                img = cv2.GaussianBlur(img, (5, 5), 0)
                img = cv2.resize(img, size)
                roi = cv2.resize(mask, size)

                # Append resized image and ROI to lists
                images.append(img)
                rois.append(roi)

        except Exception as e:
            print(f"Error processing image file {file_path}: {e}. Skipping.")

    return np.array(images), np.array(rois)

# ROI USING TEMPLATE MATCHING

In [3]:
def load_images(folder, size, template):
    images = []

    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)

        if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue

        try:
            img = Image.open(file_path).convert('L')
            img = np.array(img)
            roi = find_roi(img, template) 
            if roi is not None:
                img = roi
                img = Image.fromarray(img)
                img = img.resize(size)
                img = np.array(img).flatten()
                images.append(img)

        except UnidentifiedImageError:
            print(f"Cannot identify image file {file_path}. Skipping.")

    return images


def find_roi(image, template):
    # Apply template matching to find the ROI
    res = cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
    if max_val >= 0.5:
        top_left = max_loc
        h, w = template.shape
        bottom_right = (top_left[0] + w, top_left[1] + h)
        roi = image[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]
        return roi
    else:
        # NO ROI found
        return None

# ROI USING CONTOUR DETECTION

In [4]:
def load_images(folder, size, threshold_value):
    images = []

    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)

        if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue

        try:
            img = Image.open(file_path).convert('L')
            img = np.array(img)

            # Find ROI using the specific threshold
            roi = find_roi(img, threshold_value)
            if roi is not None:
                x, y, w, h = roi
                img = img[y:y+h, x:x+w]

                img = Image.fromarray(img)
                img = img.resize(size)
                img = np.array(img).flatten()
                images.append(img)

        except UnidentifiedImageError:
            print(f"Cannot identify image file {file_path}. Skipping.")

    return images

def find_roi(image, threshold_value):
    # Convert the image to black and white
    _, binary_image = cv2.threshold(image, threshold_value, 255, cv2.THRESH_BINARY)

    inverted_binary_image = cv2.bitwise_not(binary_image)

    # Find contours on the binary image
    contours, _ = cv2.findContours(inverted_binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        # Find the largest contour by area
        largest_contour = max(contours, key=cv2.contourArea)

        # Get the bounding box of the largest contour
        x, y, w, h = cv2.boundingRect(largest_contour)
        return (x, y, w, h)
    else:
        return None

# ROI USING MASKING

In [5]:
def load_images(folder, size, mask_ratios):
    images = []

    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)

        if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue

        try:
            img = Image.open(file_path).convert('L')
            img = np.array(img)

            # Find ROI using masking
            for mask_ratio in mask_ratios:
                roi = find_roi(img, mask_ratio)
                if roi is not None:
                    x, y, w, h = roi
                    img_roi = img[y:y+h, x:x+w]
                    img_roi = Image.fromarray(img_roi)
                    img_roi = img_roi.resize(size)
                    img_roi = np.array(img_roi).flatten()
                    images.append((img_roi, mask_ratio))

        except UnidentifiedImageError:
            print(f"Cannot identify image file {file_path}. Skipping.")

    return images

def find_roi(image, mask_ratio):
    h, w = image.shape
    x, y, w, h = int(w * mask_ratio[0]), int(h * mask_ratio[1]), int(w * mask_ratio[2]), int(h * mask_ratio[3])
    return (x, y, w, h)

# ROI USING EDGE DETECTION

In [6]:
import cv2

def load_images(folder, size):
    images = []
    rois = []

    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)

        if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue

        try:
            img = cv2.imread(file_path)
            img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            # Apply edge detection (Canny)
            edges = cv2.Canny(img_gray, 100, 200)

            # Create a mask based on edges
            mask = np.zeros_like(img_gray, dtype=np.uint8)
            mask[edges > 0] = 255

            # Apply the mask to the original colored image
            masked_img = cv2.bitwise_and(img, img, mask=mask)

            # Resize image and mask to the specified size
            img_resized = cv2.resize(masked_img, size)
            mask_resized = cv2.resize(mask, size)
            
            images.append(img_resized)
            rois.append(mask_resized)

        except Exception as e:
            print(f"Error processing image file {file_path}: {e}. Skipping.")

    return np.array(images), np.array(rois)

# COMBINING ALL THE TECHNIQUES (I.E., THE NAIVE APPROACH, CONTOUR DETECTION, AND MASKING

In [7]:
def load_images(folder, size):
    images = []
    rois = []

    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)

        if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue

        try:
            img = cv2.imread(file_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

            # Calculate difference between saturation and brightness
            diff = cv2.absdiff(img[:, :, 1], img[:, :, 2])

            # Apply threshold (adjust based on your images)
            thresh = cv2.threshold(diff, 20, 255, cv2.THRESH_BINARY)[1]

            # Median filter to remove noise
            thresh = cv2.medianBlur(thresh, 5)

            # Find contours
            contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            # Select largest contour as potential ROI
            largest_contour = None
            largest_area = 0
            for cnt in contours:
                area = cv2.contourArea(cnt)
                if area > largest_area:
                    largest_contour = cnt
                    largest_area = area

            # Extract ROI using mask if largest contour found
            if largest_contour is not None:
                mask = np.zeros_like(img[:, :, 0], dtype=np.uint8)
                cv2.drawContours(mask, [largest_contour], -1, 255, -1)
                img = cv2.bitwise_and(img, img, mask=mask)

                # Resize image and ROI to specified size
                img = cv2.cvtColor(img, cv2.COLOR_HSV2BGR)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                img = cv2.GaussianBlur(img, (5, 5), 0)
                img = cv2.resize(img, size)
                roi = cv2.resize(mask, size)

                # Append resized image and ROI to lists
                images.append(img)
                rois.append(roi)

        except:
            print(f"Error processing image file {file_path}. Skipping.")

    return np.array(images), np.array(rois)

# CNN MODEL PROPOSED

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, Reshape
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras import backend as K

image_size = (64, 64)

# Define CNN model
model = Sequential([
    Input(shape=(image_size[0], image_size[1], 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(64 * 64, activation='sigmoid'),
    Reshape((64, 64, 1))
])

# Loading the Dataset

In [11]:
from sklearn.model_selection import train_test_split

image_size = (64, 64)

male_images, male_rois = load_images('data/male', image_size) # replace with your local path
female_images, female_rois = load_images('data/female', image_size) # replace with your local path

X = np.concatenate([male_images, female_images])
y = np.concatenate([male_rois, female_rois])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0